In [3]:
import pandas as pd
import cv2 as cv
import sys 

import sklearn
import numpy as np 
import os
import pandas as pd
import time

import warnings
warnings.filterwarnings('ignore')

%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt

from IPython.utils import io

In [ ]:
sync = pd.read_csv('data/hands/Training/Shahar Terner/Sachar636771046527112972Sync.csv')
spot = pd.read_csv('data/hands/Training/Shahar Terner/Sachar636771045591258071Spontan.csv')
alone = pd.read_csv('data/hands/Training/Shahar Terner/Sachar636771047408504303Alone.csv')
alone_cmp = pd.read_csv('data/hands/HandRight.csv')


In [ ]:
alone_cmp.head()

In [ ]:
alone.head()

In [ ]:
alone_cmp['Time'] = alone_cmp['Time'] + (alone.iloc[0,0] - alone_cmp.iloc[0,0])
alone_cmp.head()

In [ ]:
sync = sync.set_index('Time')
spot = spot.set_index('Time')
alone = alone.set_index('Time')
alone_cmp = alone_cmp.set_index('Time')

In [ ]:
sync.describe()

In [ ]:
sync.info()

In [ ]:
sync.head()

In [ ]:
left_s = sync[sync[' Hand Type'] == 'left']
right_s = sync[sync[' Hand Type'] == 'right']

left_sp = spot[spot[' Hand Type'] == 'left']
right_sp = spot[spot[' Hand Type'] == 'right']

In [ ]:
plt.figure()
i = 1
for x in range(1,54 ,3):
    plt.subplot(18,3,x)
    left_s.iloc[:,i+2].plot(figsize=(25,100),label='left')
    right_s.iloc[:,i+2].plot(figsize=(25,100),label='right')
    plt.title("sync {}".format(left_s.columns[i+2]))
    plt.legend()

    plt.subplot(18,3,x + 1)
    left_sp.iloc[:,i+2].plot(figsize=(25,100),label='left')
    right_sp.iloc[:,i+2].plot(figsize=(25,100),label='right')
    plt.title("spot {}".format(left_s.columns[i+2]))
    plt.legend()
    
    plt.subplot(18,3,x + 2)
    alone.iloc[:,i+2].plot(figsize=(25,100),label='left')
    alone_cmp.iloc[:,i+2].plot(figsize=(25,100),label='right')
    plt.title("alone {}".format(left_s.columns[i+2]))
    plt.legend()

    i = i + 1
plt.show()



## Loading the data

In [42]:
right_hand = pd.read_csv('data/hands/HandRight.csv') 

right_hand[" Hand Type"] = "right"
right_hand = right_hand.dropna()

def join_right_hand(df):

    right_hand['Time'] = df['Time']
    right_hand[' Frame ID'] = df[[' Frame ID']]

    frames = [df,right_hand]
    df = pd.concat(frames)

    df = df.sort_values(' Hand Type').sort_values('Time',kind='stable')
    x = df[' Hand Type'].value_counts()
    df = df.reset_index().iloc[:x.iloc[1] * 2]
    
    df[" # hands"] = 2

    return df

In [99]:
def proccess_df(df):
    df = df.drop(df[df[" # hands"] == 1].index)
    df = df.replace(["right","left"],[0,1])
    return df

In [110]:
def load_data(path):
    li = []

    for subject in os.listdir(path):
        for file in os.listdir(path + '/' + subject):
            print(path + '/' + subject + '/' + file)
            df = pd.read_csv(path + '/' + subject + '/' + file)
            if "Alone" in file:
                df = join_right_hand(df)
                df['OUTCOME'] = 0
            elif "Sync" in file:
                df['OUTCOME'] = 1
            else:
                df['OUTCOME'] = 2
            df = proccess_df(df)
            li.append(df)
    df = pd.concat(li).iloc[:,:-1]
    return df

In [111]:
df = load_data('data/hands/Training')
df_test = load_data('data/hands/Validation')

data/hands/Training/Shahar Terner/Sachar636771045591258071Spontan.csv
data/hands/Training/Shahar Terner/Sachar636771047408504303Alone.csv
data/hands/Training/Shahar Terner/Sachar636771046527112972Sync.csv
data/hands/Training/Orya Kalmanovitz/OryaB636771082736601528Alone.csv
data/hands/Training/Orya Kalmanovitz/OryaB636771083605535985Spontan.csv
data/hands/Training/Orya Kalmanovitz/OryaB636771084494502229Sync.csv
data/hands/Training/Evyatar Cohen/Evyatar636771053639929594Sync.csv
data/hands/Training/Evyatar Cohen/Evyatar636771054555711409Alone.csv
data/hands/Training/Evyatar Cohen/Evyatar636771052727603804Spontan.csv
data/hands/Training/Shelly Social_Nuero/Shelly636759787508979097Spontan.csv
data/hands/Training/Shelly Social_Nuero/Shelly636759789526313406Sync.csv
data/hands/Training/Shelly Social_Nuero/Shelly636759788383205274Alone.csv
data/hands/Training/Yael Hagai/Yael636771060699529085Sync.csv
data/hands/Training/Yael Hagai/Yael636771061659067649Alone.csv
data/hands/Training/Yael Hag

In [109]:
df.head()

,Time,Frame ID,Hand Type,# hands,Position X,Position Y,Position Z,Velocity X,Velocity Y,Velocity Z,...,Wrist Pos Y,Wrist Pos Z,Elbow pos X,Elbow Pos Y,Elbow Pos Z,Grab Strenth,Grab Angle,Pinch Strength,OUTCOME,index
0,379.7184,46329.0,0,2,174.48970,236.7132,-34.12203,-84.29025,65.13725,124.30900,...,194.2103,6.415048,351.6485,31.24136,108.23130,0.0,0.413517,0.000000,2,NaN
1,379.7184,46329.0,1,2,-68.01415,156.5934,-14.45648,-112.72190,188.78510,61.37488,...,119.9337,34.933900,-160.2634,-96.42902,114.71420,0.0,1.152870,0.396940,2,NaN
2,379.7348,46331.0,0,2,173.27060,238.6520,-30.50214,-78.89249,98.40533,233.60370,...,195.5094,8.037281,357.1137,31.69973,100.64270,0.0,0.265857,0.000000,2,NaN
3,379.7348,46331.0,1,2,-70.25227,160.8049,-13.16562,-120.68820,248.41160,68.05241,...,120.8761,35.246690,-156.5777,-96.99940,113.46690,0.0,1.191082,0.539377,2,NaN
4,379.7516,46333.0,0,2,172.33350,241.7933,-25.97062,-51.04300,195.62840,287.99430,...,196.8314,9.881967,359.0114,31.43021,95.73518,0.0,0.232197,0.000000,2,NaN


In [105]:
df["OUTCOME"].value_counts()

2    75310
1    75278
0    74620
Name: OUTCOME, dtype: int64

In [103]:
x_train = df.drop('OUTCOME',axis=1).to_numpy()
y_train = df['OUTCOME']
x_test = df_test.drop('OUTCOME',axis=1).to_numpy()
y_test = df_test['OUTCOME']


In [94]:
df_test.head()

,Time,Frame ID,Hand Type,# hands,Position X,Position Y,Position Z,Velocity X,Velocity Y,Velocity Z,...,Wrist Pos Y,Wrist Pos Z,Elbow pos X,Elbow Pos Y,Elbow Pos Z,Grab Strenth,Grab Angle,Pinch Strength,OUTCOME,index
0,772.2799,90321.0,right,2,65.37845,224.3082,112.75900,-112.50850,-293.12840,318.10640,...,196.6624,107.62950,351.3936,90.84698,131.1359,0.0,0.453898,0.000000,2,NaN
1,772.2799,90321.0,left,2,-218.35700,305.6544,63.57877,193.24920,313.48280,-16.51894,...,279.1037,91.25497,-481.7012,163.33340,167.2560,0.0,0.550612,0.460952,2,NaN
2,772.2964,90323.0,right,2,63.55072,220.1848,117.93530,-57.90208,-37.84834,377.73690,...,193.0091,111.56650,354.6543,98.62046,135.8045,0.0,0.495044,0.000000,2,NaN
3,772.2964,90323.0,left,2,-213.44920,312.6948,62.97603,378.39140,334.78480,-62.01735,...,287.6982,89.44084,-475.7014,169.70460,168.2110,0.0,0.540570,0.128885,2,NaN
4,772.3131,90325.0,right,2,61.66813,217.5243,122.59860,-90.69568,-84.79558,298.46620,...,190.9333,115.36450,356.3591,105.17530,139.1584,0.0,0.518979,0.000000,2,NaN


In [104]:
from xgboost import XGBClassifier

xgb = XGBClassifier()
xgb.fit(x_train,y_train)
xgb.score(x_test,y_test)

[19:47:34] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


0.49528556034482757

In [106]:
from sklearn.ensemble import AdaBoostClassifier

ada = AdaBoostClassifier()
ada.fit(x_train,y_train)
ada.score(x_test,y_test)

0.5662416187739464